In [ ]:
!pip install -U layoutparser
!pip install 'git+https://github.com/facebookresearch/detectron2.git@v0.4#egg=detectron2'
!git clone https://github.com/Layout-Parser/layout-parser.git
!pip install layoutparser # Install the base layoutparser library with
!pip install "layoutparser[layoutmodels]" # Install DL layout model toolkit

In [ ]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.6 MB/s eta 0:00:00


In [ ]:
!pip install pillow

## Create a streamlit app example


In [ ]:
!pip install numpy

In [ ]:
%%writefile app.py

import streamlit as st
import layoutparser as lp
import cv2
from PIL import Image
import numpy as np

def load_image(image_file):
  img = Image.open(image_file)
  return img

st.session_state['key'] = None


st.sidebar.image("/content/logo.png")
with st.form("my_form"):
  st.title("Upload file for Data Extraction")
  upload_file = st.file_uploader("", type=["png","jpg","jpeg"])

  submitted = st.form_submit_button("Submit for Extraction")
  if submitted:

      # Load image using OpenCV
    image = cv2.imdecode(np.fromstring(upload_file.read(), np.uint8), 1)

    #image = cv2.imread(upload_file)
    image = image[..., ::-1]

    model = lp.Detectron2LayoutModel('lp://PubLayNet/faster_rcnn_R_50_FPN_3x/config',
                                    extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.8],
                                    label_map={0: "Text", 1: "Title", 2: "List", 3:"Table", 4:"Figure"})

    # Detect the layout of the input image
    layout = model.detect(image)


    # Show the detected layout of the input image
    lp.draw_box(image, layout, box_width=3)


    text_blocks = lp.Layout([b for b in layout if b.type=='Text'])
    figure_blocks = lp.Layout([b for b in layout if b.type=='Figure'])

    h, w = image.shape[:2]

    left_interval = lp.Interval(0, w/2*1.05, axis='x').put_on_canvas(image)

    left_blocks = text_blocks.filter_by(left_interval, center=True)
    left_blocks.sort(key = lambda b:b.coordinates[1], inplace=True)
    # The b.coordinates[1] corresponds to the y coordinate of the region
    # sort based on that can simulate the top-to-bottom reading order
    right_blocks = lp.Layout([b for b in text_blocks if b not in left_blocks])
    right_blocks.sort(key = lambda b:b.coordinates[1], inplace=True)

    # And finally combine the two lists and add the index
    text_blocks = lp.Layout([b.set(id = idx) for idx, b in enumerate(left_blocks + right_blocks)])

    imagelayout=lp.draw_box(image, text_blocks,
                box_width=3,
                show_element_id=True)

    ocr_agent = lp.TesseractAgent(languages='eng')
        # Initialize the tesseract ocr engine. You might need
        # to install the OCR components in layoutparser:
        # pip install layoutparser[ocr]



    for block in text_blocks:
        segment_image = (block
                          .pad(left=5, right=5, top=5, bottom=5)
                          .crop_image(image))
            # add padding in each image segment can help
            # improve robustness

        text = ocr_agent.detect(segment_image)
        block.set(text=text, inplace=True)

    l=[]
    st.success("Data Extracted from image: ")
    for txt in text_blocks.get_texts():
        #st.text(txt)
        l.append(txt)


    #st.text(l)
    ouput_text = "\n".join(l)

    file_details = {"filename":upload_file.name, "filetype":upload_file.type,
                              "filesize":upload_file.size}
    file_details["extracted_data"] = ouput_text
    st.header("Meta Data")
    st.write(file_details)



    col1, col2 = st.sidebar.columns(2)
    with col2:
      st.sidebar.header("Uploaded Image")
      st.sidebar.image(image, use_column_width=True)

    with col1:
      st.sidebar.header("Extracted text")
      st.sidebar.text(ouput_text)

    st.session_state['key'] = ouput_text

if st.session_state['key'] is not None:
  st.download_button('Download', st.session_state['key'])




Writing app.py


## Install localtunnel

In [ ]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 2.228s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details


## Run streamlit in background

In [ ]:
!streamlit run /content/app.py &>/content/logs.txt &

import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 34.136.69.244


## Expose the port 8501
Then just click in the `url` showed.

A `log.txt`file will be created.

In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 2.653s
your url is: https://long-views-act.loca.lt


[![ko-fi](https://www.ko-fi.com/img/githubbutton_sm.svg)](https://ko-fi.com/Y8Y3VYYE)